In [ ]:
%load_ext autoreload
%autoreload 2
import pickle
import sys

import dask
import dask.array as da
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from dask import dataframe as dd
from matplotlib.gridspec import GridSpec
from scipy import stats

In [ ]:
# read the df created in calculate_IPAS_area_ratio.ipynb
df_IPAS = pd.read_hdf("df_IPAS_rand_area_ratio_no_points.h5")

In [ ]:
len(df_IPAS)

In [ ]:
# read in aggregate-aggregate collection data
f = open("../instance_files/instance_db_aggagg_rand", "rb")
results = pickle.load(f)
(
    agg_as_aggagg_rand,
    agg_bs_aggagg_rand,
    agg_cs_aggagg_rand,
    phi2Ds_aggagg_rand,
    cplxs_aggagg_rand,
    dds_aggagg_rand,
) = (
    results["agg_as"],
    results["agg_bs"],
    results["agg_cs"],
    results["phi2Ds"],
    results["cplxs"],
    results["dds"],
)

f.close()

In [ ]:
# truncate df for monomer aspect ratio and number
# <=50 based on column CPI dataset (50 was max aspect ratio)
upper_bound = 70.0
trunc_IPAS3d = df_IPAS[
    (df_IPAS["mono_phi"] <= upper_bound)
]  # saving a copy of 3d aspect ratios to plot
truncall = df_IPAS[(df_IPAS["mono_phi"] <= upper_bound)]
trunc20 = df_IPAS[(df_IPAS["mono_phi"] <= upper_bound) & (df_IPAS["ncrystals"] <= 20)]
trunc10 = df_IPAS[(df_IPAS["mono_phi"] <= upper_bound) & (df_IPAS["ncrystals"] <= 10)]
trunc5 = df_IPAS[(df_IPAS["mono_phi"] <= upper_bound) & (df_IPAS["ncrystals"] <= 5)]
trunc2 = df_IPAS[(df_IPAS["mono_phi"] <= upper_bound) & (df_IPAS["ncrystals"] <= 2)]

lower_bound = 0.01
upper_bound = 10.0
trunc_outer = df_IPAS[
    (df_IPAS["mono_phi"] <= upper_bound) & (df_IPAS["mono_phi"] >= lower_bound)
]

lower_bound = 0.5
upper_bound = 2.0
trunc_inner = df_IPAS[
    (df_IPAS["mono_phi"] <= upper_bound) & (df_IPAS["mono_phi"] >= lower_bound)
]

In [ ]:
len(df_CPI), len(df_nonan), len(truncall), len(trunc_outer), len(trunc_inner)

In [ ]:
len(df_CPI), len(df_nonan), len(truncall), len(trunc_outer), len(trunc_inner)

In [ ]:
# read in CPI data
# all campaings in one file
df_CPI = pd.read_csv("df_IPAS_ellipse_aggs_allcampaigns.csv")
# only use aggregates
df_CPI = df_CPI[(df_CPI["classification"] == "agg") & (df_CPI["cutoff"] < 5)]

In [ ]:
# gather columns that are the same as IPAS dataframe to merge
df_CPI.rename(
    columns={
        "IPAS_ellipse": "Aspect Ratio",
        "filled_circular_area_ratio": "Area Ratio",
        "complexity": "Complexity",
    },
    inplace=True,
)
CPI = df_CPI[["Complexity", "Aspect Ratio", "Area Ratio"]]

truncall.rename(
    columns={"area_ratio": "Area Ratio", "cplx": "Complexity", "phi2D": "Aspect Ratio"},
    inplace=True,
)
trunc_all = truncall[["Complexity", "Aspect Ratio", "Area Ratio"]]
trunc_outer.rename(
    columns={"area_ratio": "Area Ratio", "cplx": "Complexity", "phi2D": "Aspect Ratio"},
    inplace=True,
)
trunc_outer = trunc_outer[["Complexity", "Aspect Ratio", "Area Ratio"]]
trunc_inner.rename(
    columns={"area_ratio": "Area Ratio", "cplx": "Complexity", "phi2D": "Aspect Ratio"},
    inplace=True,
)
trunc_inner = trunc_inner[["Complexity", "Aspect Ratio", "Area Ratio"]]

In [ ]:
df_cplx = (
    pd.concat(
        [
            truncall["Complexity"],
            trunc_outer["Complexity"],
            trunc_inner["Complexity"],
            CPI["Complexity"],
        ],
        keys=["$\phi_m$=0.01-70.0", "$\phi_m$=0.1-10.0", "$\phi_m$=0.5-2.0", "CPI"],
        names=["Source"],
    )
    .reset_index()
    .drop(columns="level_1")
)
df_phi = (
    pd.concat(
        [
            truncall["Aspect Ratio"],
            trunc_outer["Aspect Ratio"],
            trunc_inner["Aspect Ratio"],
            CPI["Aspect Ratio"],
        ],
        keys=["$\phi_m$=0.01-70.0", "$\phi_m$=0.1-10.0", "$\phi_m$=0.5-2.0", "CPI"],
        names=["Source"],
    )
    .reset_index()
    .drop(columns="level_1")
)
df_ar = (
    pd.concat(
        [
            truncall["Area Ratio"],
            trunc_outer["Area Ratio"],
            trunc_inner["Area Ratio"],
            CPI["Area Ratio"],
        ],
        keys=["$\phi_m$=0.01-70.0", "$\phi_m$=0.1-10.0", "$\phi_m$=0.5-2.0", "CPI"],
        names=["Source"],
    )
    .reset_index()
    .drop(columns="level_1")
)

In [ ]:
# combine CPI and IPAS data to plot
# aspect ratio only for cutoff > 5%, otherwise NaN
cdf_all = (
    pd.concat(
        [truncall, trunc_outer, trunc_inner, CPI],
        keys=["$\phi_m$=0.01-70.0", "$\phi_m$=0.1-10.0", "$\phi_m$=0.5-2.0", "CPI"],
        names=["Source"],
    )
    .reset_index()
    .drop(columns="level_1")
)

# cdf_outer = pd.concat([CPI, trunc_outer], keys=['CPI', 'IPAS'], names=["Source"]).reset_index().drop(columns='level_1')

# cdf_inner = pd.concat([CPI, trunc_inner], keys=['CPI', 'IPAS'], names=["Source"]).reset_index().drop(columns='level_1')

In [ ]:
# for plotting. need a source column for hue argument in seaborn
cdf_all = pd.melt(
    cdf_all,
    id_vars=["Source"],
    value_vars=["Complexity", "Aspect Ratio", "Area Ratio"],
    value_name="Magnitude",
)
# cdf_outer = pd.melt(cdf_outer,id_vars=['Source'],value_vars=['Complexity', 'Aspect Ratio', 'Area Ratio'],
#              value_name='Value')
# cdf_inner = pd.melt(cdf_inner,id_vars=['Source'],value_vars=['Complexity', 'Aspect Ratio', 'Area Ratio'],
#              value_name='Value')

In [ ]:
df_nonan = df_CPI[df_CPI["Aspect Ratio"].notnull()]

In [ ]:
# bins, edges = np.histogram(np.isfinite(truncall['phi2D']), density=True, bins = 70)
n, bins, patchs = plt.hist(
    truncall["phi2D"][np.isfinite(truncall["phi2D"])], density=True, bins=70
);

In [ ]:
n, bins, patchs = plt.hist(
    truncall["area ratio"][np.isfinite(truncall["area ratio"])], density=True, bins=70
);

In [ ]:
n, bins, patchs = plt.hist(CPI["Area Ratio"], density=True, bins=70)
plt.axvline(x=0.44, color="magenta", label="RMSE 1")

In [ ]:
len(cplxs_aggagg_rand[:, :, :].flatten())

In [ ]:
xlarge = 24
large = 20
med = 16
small = 12
params = {
    "axes.titlesize": xlarge,
    "legend.fontsize": med,
    "axes.labelsize": xlarge,
    "xtick.labelsize": large,
    "ytick.labelsize": large,
    "figure.titlesize": large,
    "font.family": "serif",
}
import matplotlib as mpl

mpl.rc("font", family="serif")
plt.rcParams.update(params)

In [ ]:
stats.mode(df_nonan["IPAS_ellipse"])

In [ ]:
stats.mode(trunc5["cplx"])

In [ ]:
# fig, ax = plt.subplots()
f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
sns.set_style("white")

g = sns.distplot(
    df_nonan["Aspect Ratio"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "color": "#FA2400"},
    label="CPI aggregates",
    ax=ax2,
)

# sns.distplot(df_nonan['opencv_ellipse'], hist = False, kde = True, norm_hist=True,
#              kde_kws = {'linewidth': 3,'color':'darkred'}, label='CPI (opencv ellipse)', ax=ax2)

g = sns.distplot(
    truncall["phi2D"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "color": "lightskyblue"},
    label="IPAS $\phi_{2D}$ $n_m$ $\leq$ 30",
    ax=ax2,
)

g = sns.distplot(
    trunc20["phi2D"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "linestyle": "--", "color": "lightskyblue"},
    label="IPAS $\phi_{2D}$ $n_m$ $\leq$ 20",
    ax=ax2,
)

g = sns.distplot(
    trunc10["phi2D"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "linestyle": "-.", "color": "lightskyblue"},
    label="IPAS $\phi_{2D}$ $n_m$ $\leq$ 10",
    ax=ax2,
)

# g=sns.distplot(trunc5['phi2D'], hist = False, kde = True, norm_hist=True,
#             kde_kws = {'linewidth': 3, 'linestyle':':','color':'lightskyblue'}, label='IPAS $\phi_{2D}$ $n_m$ < 5',ax=ax2)

g = sns.distplot(
    truncall["agg_phi"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "color": "#0B1CE0"},
    label="IPAS $\phi_{ca}$ $n_m$ $\leq$ 30",
    ax=ax2,
)

g = sns.distplot(
    trunc20["agg_phi"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "linestyle": "--", "color": "#0B1CE0"},
    label="IPAS $\phi_{ca}$ $n_m$ $\leq$ 20",
    ax=ax2,
)

g = sns.distplot(
    trunc10["agg_phi"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "linestyle": "-.", "color": "#0B1CE0"},
    label="IPAS $\phi_{ca}$ $n_m$ $\leq$ 10",
    ax=ax2,
)

# g=sns.distplot(trunc5['agg_phi'], hist = False, kde = True, norm_hist=True,
#             kde_kws = {'linewidth': 3, 'linestyle':':','color':'#0B1CE0'}, label='IPAS $\phi_{ca}$ $n_m$ < 5',ax=ax2)
# g=sns.distplot(phi2Ds_aggagg_rand[1:11,:,:], hist = False, kde = True, norm_hist=True,
#             kde_kws = {'linewidth': 3, 'linestyle':'-.','color':'green'}, label='IPAS agg-agg',ax=ax2)


ax2.set_xlabel("Aspect Ratio", fontsize=18)
ax2.set_ylabel("Frequency", fontsize=18)
ax2.set_ylim(0, 5)
ax2.set_xlim(0.0, 1.0)
ax2.legend(loc="upper center", ncol=2)

sns.distplot(
    df_CPI["Complexity"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "color": "red"},
    label="CPI aggregates",
    ax=ax1,
)
sns.distplot(
    truncall["cplx"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "color": "#0B1CE0"},
    label="IPAS $n_m$ $\leq$ 30",
    ax=ax1,
)
sns.distplot(
    trunc20["cplx"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "linestyle": "--", "color": "#0B1CE0"},
    label="IPAS $n_m$ $\leq$ 20",
    ax=ax1,
)
sns.distplot(
    trunc10["cplx"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "linestyle": "-.", "color": "#0B1CE0"},
    label="IPAS $n_m$ $\leq$ 10",
    ax=ax1,
)
sns.distplot(
    trunc5["cplx"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "linestyle": ":", "color": "#0B1CE0"},
    label="IPAS $n_m$ $\leq$ 5",
    ax=ax1,
)
# sns.distplot(cplxs_aggagg_rand[:,:,:].flatten(), hist = False, kde = True, norm_hist=True,
#              kde_kws = {'linewidth': 3, 'linestyle':'-.','color':'green'}, label='IPAS agg-agg', ax=ax1)

ax1.set_xlabel("Complexity", fontsize=18)
ax1.set_ylabel("Frequency", fontsize=18)
ax1.set_xlim(0.0, 1.0)
ax1.set_ylim(0, 8)
ax1.legend()

sns.distplot(
    df_CPI["Area Ratio"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "color": "red"},
    label="CPI aggregates",
    ax=ax3,
)
sns.distplot(
    truncall["area_ratio"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "color": "#0B1CE0"},
    label="IPAS $n_m$ $\leq$ 30",
    ax=ax3,
)
sns.distplot(
    trunc20["area_ratio"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "linestyle": "--", "color": "#0B1CE0"},
    label="IPAS $n_m$ $\leq$ 20",
    ax=ax3,
)
sns.distplot(
    trunc10["area_ratio"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "linestyle": "-.", "color": "#0B1CE0"},
    label="IPAS $n_m$ $\leq$ 10",
    ax=ax3,
)

ax3.set_xlabel("Area Ratio", fontsize=18)
ax3.set_ylabel("Frequency", fontsize=18)
ax3.set_xlim(0.0, 1.0)
ax3.set_ylim(0, 5)
ax3.legend()

ax4 = sns.boxplot(
    x="variable",
    y="Value",
    hue="Source",
    data=cdf,
    palette={"CPI": "red", "IPAS": "#0B1CE0"},
    showfliers=False,
    ax=ax4,
)

for i in range(0, 6, 2):
    cpi_color = ax4.artists[i]
    cpi_color.set_facecolor("red")
for i in range(1, 6, 2):
    ipas_color = ax4.artists[i]
    ipas_color.set_facecolor("#0B1CE0")
ax4.set_xlabel("Bulk Statistics", fontsize=18)
# ax4.legend()

# plt.savefig('../plots/CPI_IPAS_distributions.png', dpi=300, bbox_inches='tight')

In [ ]:
# Setting up the plot surface
fig = plt.figure(figsize=(22, 15))
gs = GridSpec(nrows=2, ncols=3, wspace=0.1)

# First axes
ax1 = fig.add_subplot(gs[0, 0])
sns.distplot(
    truncall["Complexity"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "color": "#0D007E"},
    label="IPAS $n_m$ $\leq$ 30",
    ax=ax1,
)
sns.distplot(
    trunc20["cplx"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "linestyle": "--", "color": "#0D007E"},
    label="IPAS $n_m$ $\leq$ 20",
    ax=ax1,
)
sns.distplot(
    trunc10["cplx"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "linestyle": "-.", "color": "#0D007E"},
    label="IPAS $n_m$ $\leq$ 10",
    ax=ax1,
)
sns.distplot(
    trunc5["cplx"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "linestyle": ":", "color": "#0D007E"},
    label="IPAS $n_m$ $\leq$ 5",
    ax=ax1,
)
# sns.distplot(cplxs_aggagg_rand[:,:,:].flatten(), hist = False, kde = True, norm_hist=True,
#              kde_kws = {'linewidth': 3, 'linestyle':'-.','color':'green'}, label='IPAS agg-agg', ax=ax1)
sns.distplot(
    df_CPI["Complexity"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "color": "red"},
    label="CPI",
    ax=ax1,
)

ax1.set_xlabel("Complexity", fontsize=18)
ax1.set_ylabel("Frequency", fontsize=18)
ax1.set_xlim(0.0, 1.0)
ax1.set_ylim(0, 8)
ax1.legend()

# Second axes
ax2 = fig.add_subplot(gs[0, 1])

# sns.distplot(df_nonan['opencv_ellipse'], hist = False, kde = True, norm_hist=True,
#              kde_kws = {'linewidth': 3,'color':'darkred'}, label='CPI (opencv ellipse)', ax=ax2)

g = sns.distplot(
    truncall["Aspect Ratio"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "color": "#a1ceff"},
    label="IPAS $\phi_{2D}$ $n_m$ $\leq$ 30",
    ax=ax2,
)

g = sns.distplot(
    trunc20["phi2D"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "linestyle": "--", "color": "#a1ceff"},
    label="IPAS $\phi_{2D}$ $n_m$ $\leq$ 20",
    ax=ax2,
)

g = sns.distplot(
    trunc10["phi2D"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "linestyle": "-.", "color": "#a1ceff"},
    label="IPAS $\phi_{2D}$ $n_m$ $\leq$ 10",
    ax=ax2,
)

# g=sns.distplot(trunc5['phi2D'], hist = False, kde = True, norm_hist=True,
#             kde_kws = {'linewidth': 3, 'linestyle':':','color':'lightskyblue'}, label='IPAS $\phi_{2D}$ $n_m$ < 5',ax=ax2)

g = sns.distplot(
    trunc_IPAS3d["agg_phi"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "color": "#0D007E"},
    label="IPAS $\phi_{ca}$ $n_m$ $\leq$ 30",
    ax=ax2,
)

g = sns.distplot(
    trunc20["agg_phi"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "linestyle": "--", "color": "#0D007E"},
    label="IPAS $\phi_{ca}$ $n_m$ $\leq$ 20",
    ax=ax2,
)

g = sns.distplot(
    trunc10["agg_phi"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "linestyle": "-.", "color": "#0D007E"},
    label="IPAS $\phi_{ca}$ $n_m$ $\leq$ 10",
    ax=ax2,
)
g = sns.distplot(
    df_nonan["Aspect Ratio"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "color": "#FA2400"},
    label="CPI",
    ax=ax2,
)

ax2.set_xlabel("Aspect Ratio", fontsize=18)
ax2.axes.get_yaxis().set_visible(False)
ax2.yaxis.set_label_text(" ")
ax2.set_ylim(0, 5)
ax2.set_xlim(0.0, 1.0)
ax2.legend(loc="upper center", ncol=2, fontsize=14)

ax3 = fig.add_subplot(gs[0, 2])
sns.distplot(
    truncall["Area Ratio"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "color": "#0D007E"},
    label="IPAS $n_m$ $\leq$ 30",
    ax=ax3,
)
sns.distplot(
    trunc20["area_ratio"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "linestyle": "--", "color": "#0D007E"},
    label="IPAS $n_m$ $\leq$ 20",
    ax=ax3,
)
sns.distplot(
    trunc10["area_ratio"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "linestyle": "-.", "color": "#0D007E"},
    label="IPAS $n_m$ $\leq$ 10",
    ax=ax3,
)
sns.distplot(
    df_CPI["Area Ratio"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "color": "red"},
    label="CPI",
    ax=ax3,
)

ax3.set_xlabel("Area Ratio", fontsize=18)
ax3.axes.get_yaxis().set_visible(False)
ax3.yaxis.set_label_text(" ")
ax3.set_xlim(0.0, 1.0)
ax3.set_ylim(0, 5)
ax3.legend()

# cdf1=0.01-100.0
# cdf2=0.1-10.0
# cdf3=0.5-2.0

ax4 = fig.add_subplot(gs[1, :])
ax4 = sns.boxplot(
    x="variable",
    y="Magnitude",
    hue="Source",
    data=cdf_all,
    palette={
        "CPI": "red",
        "IPAS $\phi_m$=0.01-70.0": "#0D007E",
        "IPAS $\phi_m$=0.1-10.0": "#2166AC",
        "IPAS $\phi_m$=0.5-2.0": "#92C5DE",
    },
    showfliers=False,
    ax=ax4,
)
ax4.set_xlabel(" ")
plt.ylabel("Magnitude", size=18)
# ax4.set_ylabel("Magnitude", labelsize=18)
ax4.tick_params(axis="both", which="major", labelsize=18)
ax4.legend().texts[0].set_text("IPAS $\phi_m$=0.01-70.0")

# handles, labels = ax4.get_legend_handles_labels()
# ax4.legend(handles=handles[1:], labels=labels[])

plt.savefig("../plots/CPI_IPAS_distributions.png", dpi=300, bbox_inches="tight")

In [ ]:
# Setting up the plot surface
fig = plt.figure(figsize=(21.5, 12.5))
gs = GridSpec(nrows=2, ncols=3, wspace=0.1)

# First axes
ax1 = fig.add_subplot(gs[0, 0])
sns.distplot(
    truncall["Complexity"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "color": "#0D007E"},
    label="IPAS $n_m$ $\leq$ 30",
    ax=ax1,
)
sns.distplot(
    trunc20["cplx"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "linestyle": "--", "color": "#0D007E"},
    label="IPAS $n_m$ $\leq$ 20",
    ax=ax1,
)
sns.distplot(
    trunc10["cplx"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "linestyle": "-.", "color": "#0D007E"},
    label="IPAS $n_m$ $\leq$ 10",
    ax=ax1,
)
sns.distplot(
    trunc5["cplx"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "linestyle": ":", "color": "#0D007E"},
    label="IPAS $n_m$ $\leq$ 5",
    ax=ax1,
)
# sns.distplot(cplxs_aggagg_rand[:,:,:].flatten(), hist = False, kde = True, norm_hist=True,
#              kde_kws = {'linewidth': 3, 'linestyle':'-.','color':'green'}, label='IPAS agg-agg', ax=ax1)
sns.distplot(
    df_CPI["Complexity"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "color": "red"},
    label="CPI",
    ax=ax1,
)

ax1.set_xlabel("Complexity")
ax1.set_ylabel("Frequency")
ax1.set_ylim(0, 5)
ax1.legend(loc="upper left", ncol=1)

# Second axes
ax2 = fig.add_subplot(gs[0, 1])

# sns.distplot(df_nonan['opencv_ellipse'], hist = False, kde = True, norm_hist=True,
#              kde_kws = {'linewidth': 3,'color':'darkred'}, label='CPI (opencv ellipse)', ax=ax2)

g = sns.distplot(
    truncall["Aspect Ratio"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "color": "#a1ceff"},
    label="IPAS $\phi_{2D}$ $n_m$ $\leq$ 30",
    ax=ax2,
)

g = sns.distplot(
    trunc20["phi2D"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "linestyle": "--", "color": "#a1ceff"},
    label="IPAS $\phi_{2D}$ $n_m$ $\leq$ 20",
    ax=ax2,
)

g = sns.distplot(
    trunc10["phi2D"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "linestyle": "-.", "color": "#a1ceff"},
    label="IPAS $\phi_{2D}$ $n_m$ $\leq$ 10",
    ax=ax2,
)

# g=sns.distplot(trunc5['phi2D'], hist = False, kde = True, norm_hist=True,
#             kde_kws = {'linewidth': 3, 'linestyle':':','color':'lightskyblue'}, label='IPAS $\phi_{2D}$ $n_m$ < 5',ax=ax2)

g = sns.distplot(
    trunc_IPAS3d["agg_phi"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "color": "#0D007E"},
    label="IPAS $\phi_{ca}$ $n_m$ $\leq$ 30",
    ax=ax2,
)

g = sns.distplot(
    trunc20["agg_phi"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "linestyle": "--", "color": "#0D007E"},
    label="IPAS $\phi_{ca}$ $n_m$ $\leq$ 20",
    ax=ax2,
)

g = sns.distplot(
    trunc10["agg_phi"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "linestyle": "-.", "color": "#0D007E"},
    label="IPAS $\phi_{ca}$ $n_m$ $\leq$ 10",
    ax=ax2,
)

g = sns.distplot(
    df_nonan["Aspect Ratio"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "color": "#FA2400"},
    label="CPI",
    ax=ax2,
)

ax2.set_xlabel("Aspect Ratio")
ax2.axes.get_yaxis().set_visible(False)
ax2.yaxis.set_label_text(" ")
ax2.set_ylim(0, 5)
ax2.legend(loc="upper center", ncol=2, fontsize=14)

ax3 = fig.add_subplot(gs[0, 2])
sns.distplot(
    truncall["Area Ratio"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "color": "#0D007E"},
    label="IPAS $n_m$ $\leq$ 30",
    ax=ax3,
)
sns.distplot(
    trunc20["area_ratio"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "linestyle": "--", "color": "#0D007E"},
    label="IPAS $n_m$ $\leq$ 20",
    ax=ax3,
)
sns.distplot(
    trunc10["area_ratio"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "linestyle": "-.", "color": "#0D007E"},
    label="IPAS $n_m$ $\leq$ 10",
    ax=ax3,
)
sns.distplot(
    df_CPI["Area Ratio"],
    hist=False,
    kde=True,
    norm_hist=True,
    kde_kws={"linewidth": 3, "color": "red"},
    label="CPI",
    ax=ax3,
)

ax3.set_xlabel("Area Ratio")
ax3.axes.get_yaxis().set_visible(False)
ax3.yaxis.set_label_text(" ")
ax3.set_ylim(0, 5)
ax3.legend()

ax4 = fig.add_subplot(gs[1, 0])
ax4 = sns.boxplot(
    x="Complexity",
    y="Source",
    data=df_cplx,
    palette={
        "CPI": "red",
        "$\phi_m$=0.01-70.0": "#0D007E",
        "$\phi_m$=0.1-10.0": "#2166AC",
        "$\phi_m$=0.5-2.0": "#92C5DE",
    },
    showfliers=False,
    ax=ax4,
    orient="h",
)

ax4.set_ylabel(" ")
ax4.set_xlabel("Complexity")
ax4.set_yticklabels(ax4.get_yticklabels(), size=20)

ax5 = fig.add_subplot(gs[1, 1])
ax5 = sns.boxplot(
    x="Aspect Ratio",
    y="Source",
    data=df_phi,
    palette={
        "CPI": "red",
        "$\phi_m$=0.01-70.0": "#0D007E",
        "$\phi_m$=0.1-10.0": "#2166AC",
        "$\phi_m$=0.5-2.0": "#92C5DE",
    },
    showfliers=False,
    ax=ax5,
    orient="h",
)

ax5.axes.get_yaxis().set_visible(False)
ax5.set_xlabel("Aspect Ratio")

ax6 = fig.add_subplot(gs[1, 2])
ax6 = sns.boxplot(
    x="Area Ratio",
    y="Source",
    data=df_ar,
    palette={
        "CPI": "red",
        "$\phi_m$=0.01-70.0": "#0D007E",
        "$\phi_m$=0.1-10.0": "#2166AC",
        "$\phi_m$=0.5-2.0": "#92C5DE",
    },
    showfliers=False,
    ax=ax6,
    orient="h",
)

ax6.axes.get_yaxis().set_visible(False)
ax6.set_xlabel("Area Ratio")

for ax in plt.gcf().axes:
    ax.set_xlim(0.0, 1.0)


plt.savefig("../plots/CPI_IPAS_distributions.png", dpi=300, bbox_inches="tight")

In [ ]:
np.std(df_nonan["Aspect Ratio"]), np.std(truncall["Aspect Ratio"])

In [ ]:
bins, edges = np.histogram(trunc5["cplx"], density=True, bins=70)
edges[np.where(bins == np.max(bins))]

CPI aggregates a little less complex 

CPI data looks more quasi-spherical in terms of aspect ratios: add in other particle types, not just aggs